# TCC - Victor Gonçalves
## Instruções de obtenção de dados
Autor: Prof. Dr. Reginaldo Gonçalves Leão Junior


### Verificação do status INMET

Consiste na chamada à função `statusINMET()` que verifica a disponibilidade de séries anuais climatológicas no recurso `/dadoshistoricos` do portal, exibe um compilado deste estatus no terminal e retorna um dicionário com a estrutura:

```
Dict{Int16, String} with n entries:
  "20XX" => "https://portal.inmet.gov.br/uploads/dadoshistoricos/20XX.zip"
```

no qual as chaves são os anos disponíveis na base no formato de um inteiro de 32 bits e os valores uma String contendo o link para download do zip file das séries para o respectivo ano.

Antes de qualquer verificação, deve-se fazer a instalação do pacote em desenvolvimento, ou verificação por atualizações.
Sendo o primeiro uso do pacote em uma determinada instalção deve-se executar as instruções do comentário `i)`, nas demais, atualizações devem ser checadas por meio das instruções do comentário `ii)`. A instrução `using Pkg` é apenas uma chamada ao módulo `Pkg` responsável pela gerência dos pacotes.

In [27]:
# i) Na primeira execução do pacote
using Pkg
#Pkg.add(url="https://github.com/GESESC/SolarPowerForecastingIFMG.jl")
#Pkg.add("DataFrames")
Pkg.add("CairoMakie")

# ii) A cada nova adição de funcionalidade
#using Pkg
#Pkg.update("SolarPowerForecastingIFMG")

   Resolving package versions...
    Updating `~/.julia/environments/v1.7/Project.toml`
  [13f3f980] + CairoMakie v0.8.7
    Updating `~/.julia/environments/v1.7/Manifest.toml`
  [159f3aea] + Cairo v1.0.5
  [13f3f980] + CairoMakie v0.8.7
  [36c8627f] + Pango_jll v1.50.3+0


Após a instalação ou atualização o pacote deve ser carregado no *kernel* em execução, em seguida a verificação do status no INMET pode ser imediatamente realizada.

In [2]:
using SolarPowerForecastingIFMG: statusINMET, obter_dados
using DataFrames

In [3]:
fonte_dados = statusINMET();

Estrato de dados INMET
Primeiro ano disponível	->	2000
Último ano disponível	->	2022


Após a execução a variável `fonte_dados` é um dicionário que contém os links para a obtenção das tabelas CSV de cada ano.

In [4]:
@show fonte_dados[2000];
@show fonte_dados[2010];
@show fonte_dados[2022];

fonte_dados[2000] = "https://portal.inmet.gov.br/uploads/dadoshistoricos/2000.zip"
fonte_dados[2010] = "https://portal.inmet.gov.br/uploads/dadoshistoricos/2010.zip"
fonte_dados[2022] = "https://portal.inmet.gov.br/uploads/dadoshistoricos/2022.zip"


### Raspagem de Dados
A extração de dados em si se dá por meio da função `obter_dados` cuja documentação pode ser vista àbaixo:

In [5]:
?obter_dados

search:



Recebe:

i) fonte_dados::Dict{Int16, String}:  um dicionário com chaves do tipo Int16, contendo o ano de uma determinada série  histórica do INMET e uma String com o link de download da série no servidor

ii) cidades::Vector{String}: um vetor contendo uma lista das cidades a serem estudadas

iii) [intervtemp::Vector{Int}] ou [intervtemp::UnitRange]:  um vetor contendo uma lista dos anos a serem estudados ou um UnitRang com este  intervalo de estudo.


No exemplo abaixo utilizamos a função `obter_dados` para a `fonte_dados` requerendo estes para a cidade de Formiga e o ano de 2010.
Note como esses parâmetros foram passados na forma de vetores, de forma tal que, mais de uma cidade, ou mesmo, mais que um ano de estudo podem ser selcionados. 

In [6]:
dados = obter_dados(fonte_dados, ["FORMIGA"], [2010]);

Excessive output truncated after 524296 bytes.

In [7]:
dados

SolarPowerForecastingIFMG.SerieCidades(Union{Nothing, SolarPowerForecastingIFMG.EstruturaDeCaptura}[SolarPowerForecastingIFMG.EstruturaDeCaptura("FORMIGA", 2010, 8760×20 DataFrame
  Row │ DATA (YYYY-MM-DD)  HORA (UTC)  PRECIPITA\xc7\xc3O TOTAL, HOR\xc…  PRES ⋯
      │ Dates.Date         Dates.Time  Float64                            Floa ⋯
──────┼─────────────────────────────────────────────────────────────────────────
    1 │ 2010-01-01         00:00:00                                  0.0       ⋯
    2 │ 2010-01-01         01:00:00                                  0.0
    3 │ 2010-01-01         02:00:00                                  0.0
    4 │ 2010-01-01         03:00:00                                  0.0
    5 │ 2010-01-01         04:00:00                                  0.0       ⋯
    6 │ 2010-01-01         05:00:00                                  0.0
    7 │ 2010-01-01         06:00:00                                  0.0
    8 │ 2010-01-01         07:00:00               

Quando `dados` é exibido, se vê um problema de codificação no nome das colunas como por exemplo `PRECIPITA\xc7\xc3O` na terceira coluna, isto é uma falha de codificação na fonte dos dados. 
A lista de nomes das colunas pode ser inspecioanda utilizando-se a função name no objeto do tipo `dataset` pertencente à `mutable struct EstruturaDeCaptura`.

In [8]:
names(dados.serie[1].dataset)

20-element Vector{String}:
 "DATA (YYYY-MM-DD)"
 "HORA (UTC)"
 "PRECIPITA\xc7\xc3O TOTAL, HOR\xc1RIO (mm)"
 "PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)"
 "PRESS\xc3O ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)"
 "PRESS\xc3O ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)"
 "RADIACAO GLOBAL (KJ/m\xb2)"
 "TEMPERATURA DO AR - BULBO SECO, HORARIA (\xb0C)"
 "TEMPERATURA DO PONTO DE ORVALHO (\xb0C)"
 "TEMPERATURA M\xc1XIMA NA HORA ANT. (AUT) (\xb0C)"
 "TEMPERATURA M\xcdNIMA NA HORA ANT. (AUT) (\xb0C)"
 "TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (\xb0C)"
 "TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (\xb0C)"
 "UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)"
 "UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)"
 "UMIDADE RELATIVA DO AR, HORARIA (%)"
 "VENTO, DIRE\xc7\xc3O HORARIA (gr) (\xb0 (gr))"
 "VENTO, RAJADA MAXIMA (m/s)"
 "VENTO, VELOCIDADE HORARIA (m/s)"
 "Column20"

A estrtura de captura de dados no submódulo `RaspagemDeDadosINMET.jl` é organizada na forma:
```Julia
mutable struct EstruturaDeCaptura
    cidade::Union{String, Nothing}
    ano::Union{Int, String, Nothing}
    dataset::Union{DataFrame, Nothing}
end

mutable struct SerieCidades 
    serie::AbstractVector{EstruturaDeCaptura}
end
```
Na qual a `mutable struct SerieCidades` é utilizada para armazenar os dados das múltiplas cidades desejadas para o estudo, e a `mutable struct EstruturaDeCaptura` cotém o nome de cada cidade, e o nome de cada ano de estudo, além dos dados proprieamente ditos na forma de `DataFrame` do tipo `DataFrames.jl`.

Em nosso caso, as seguintes colunas precisam de ajuste de codificação de caractere:

3. `PRECIPITA\xc7\xc3O TOTAL, HOR\xc1RIO (mm)` para `PRECIPITACAO TOTAL, HORARIA (mm)`;
5. `PPRESS\xc3O ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)` para `PRESSAO ATMOSFERICA MAX.NA HORA ANT.(AUT) (mB)`;
6. `PPRESS\xc3O ATMOSFERICA MIN.NA HORA ANT. (AUT) (mB)` para `PRESSAO ATMOSFERICA MIN.NA HORA ANT.(AUT) (mB)`;
7. `RADIACAO GLOBAL (KJ/m\xb2)` para `RADIACAO GLOBAL (KJ/m²)`;
8. `TEMPERATURA DO AR - BULBO SECO, HORARIA (\xb0C)` para `TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)`;
9. `TEMPERATURA DO PONTO DE ORVALHO (\xb0C)` para `TEMPERATURA DO PONTO DE ORVALHO (°C)`;
10. `TEMPERATURA M\xc1XIMA NA HORA ANT. (AUT) (\xb0C)` para `TEMPERATURA MAXIMA NA HORA ANT.(AUT) (°C)`;
11. `TEMPERATURA M\xcdNIMA NA HORA ANT. (AUT) (\xb0C)` para `TEMPERATURA MINIMA NA HORA ANT.(AUT) (°C)`;
12. `TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (\xb0C)` para `TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)`;
13. `TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (\xb0C)` para `TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)`;
18. `VENTO, DIRE\xc7\xc3O HORARIA (gr) (\xb0 (gr))` para `VENTO, DIRECAO HORARIA (gr) (°(gr))`;

Para automatizar o processo cria-se um dicionário com os índices das colunas a serem modificadas e os novos nomes seguido do ajuste iterativo sobre `dados.serie[1].dataset`

In [9]:
columns_change = (
    3=>"PRECIPITACAO TOTAL, HORARIA (mm)",
    5=>"PRESSAO ATMOSFERICA MAX.NA HORA ANT.(AUT) (mB)",
    6=>"PRESSAO ATMOSFERICA MIN.NA HORA ANT.(AUT) (mB)",
    7=>"RADIACAO GLOBAL (KJ/m²)",
    8=>"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",
    9=>"TEMPERATURA DO PONTO DE ORVALHO (°C)",
    10=>"TEMPERATURA MAXIMA NA HORA ANT.(AUT) (°C)",
    11=>"TEMPERATURA MINIMA NA HORA ANT.(AUT) (°C)",
    12=>"TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)",
    13=>"TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)",
    17=>"VENTO, DIRECAO HORARIA (gr) (°(gr))"
)
for col in columns_change
       rename!(dados.serie[1].dataset, col)
end

Evento que corrije o problema de codificação conforme pode ser verificado pelo código abaixo. Alguma abordagem funcional usando `map` ou `foreach` também pode ser usada, inclusive talvez até com maior desempenho. 

In [10]:
names(dados.serie[1].dataset)

20-element Vector{String}:
 "DATA (YYYY-MM-DD)"
 "HORA (UTC)"
 "PRECIPITACAO TOTAL, HORARIA (mm)"
 "PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)"
 "PRESSAO ATMOSFERICA MAX.NA HORA ANT.(AUT) (mB)"
 "PRESSAO ATMOSFERICA MIN.NA HORA ANT.(AUT) (mB)"
 "RADIACAO GLOBAL (KJ/m²)"
 "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)"
 "TEMPERATURA DO PONTO DE ORVALHO (°C)"
 "TEMPERATURA MAXIMA NA HORA ANT.(AUT) (°C)"
 "TEMPERATURA MINIMA NA HORA ANT.(AUT) (°C)"
 "TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)"
 "TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)"
 "UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)"
 "UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)"
 "UMIDADE RELATIVA DO AR, HORARIA (%)"
 "VENTO, DIRECAO HORARIA (gr) (°(gr))"
 "VENTO, RAJADA MAXIMA (m/s)"
 "VENTO, VELOCIDADE HORARIA (m/s)"
 "Column20"

## Aquisições múltiplas
A função `obter_dados` ainda possibilita a aquisição múltipla de dados, tanto para múltiplas cidades, quanto para distintos intervalos de tempo, veja os exemplos abaixos.

### Aquisição de dados para múltiplas cidades
Na caso da aquisição para várias cidades basta declarar quais aquelas desejadas para estudo no segundo parâmetro da função, como no exemplo abaixo. É importante considerar que as cidades declaras devem constar dentre aquelas onde o INMET mantém coleta de dados. Futuramente a função `statusINMET` checará essa informações de maneira automatizada para disponibilizar aos usuários.

In [11]:
dados = obter_dados(fonte_dados, ["FORMIGA", "BAMBUI"], [2020]);

Excessive output truncated after 524348 bytes.

Neste caso veja como os dados ficam organizados em `dados`.
Como já dito a variável é do tipo `SerieCidades` e possui o campo `serie` que contém um vetor do tipo `Vector{SolarPowerForecastingIFMG.EstruturaDeCaptura}`.

In [12]:
@show typeof(dados)
@show typeof(dados.serie)

typeof(dados) = SolarPowerForecastingIFMG.SerieCidades
typeof(dados.serie) = Vector{Union{Nothing, SolarPowerForecastingIFMG.EstruturaDeCaptura}}


Vector{Union{Nothing, EstruturaDeCaptura}} (alias for Array{Union{Nothing, SolarPowerForecastingIFMG.EstruturaDeCaptura}, 1})

O comprimento do vetor armazenado no campo `serie` pode ser exibido fazendo-se:

In [13]:
length(dados.serie)

2

Onde o primeiro elemento se refere à primeira cidade declarada em `obter_dados` e o segundo à segunda.

In [14]:
println("Primeira cidade: ", dados.serie[1].cidade)
println("Ano da série: ", dados.serie[1].ano)

println("Segunda cidade: ", dados.serie[2].cidade)
println("Ano da série: ", dados.serie[2].ano)

Primeira cidade: FORMIGA
Ano da série: 2020
Segunda cidade: BAMBUI
Ano da série: 2020


### Aquisição de dados para múltiplos períodos

A aquisição para diversos anos está habilitada em dois modos distintos, o primeiro deles, declarando o 
terceiro parâmetro de `obter_dados` como um vetor com vários elementos como no exemplo abaixo:
```Julia
dados = obter_dados(fonte_dados, cidades, intervtemp=[2019, 2020, 2021, 2022])
```

Digamos que se queira obter os dados metereológicos para a cidade de Formiga nos anos 2010, 2015 e 2020, isto pode ser feito como:

In [15]:
dados = obter_dados(fonte_dados, ["FORMIGA"], [2010, 2015, 2020]);

Excessive output truncated after 524302 bytes.

As múltiplas séries exigem que a correção de codificação seja feita para cada ano o que exige um *loop* superior àquele mostrado anteriormente.

In [16]:
columns_change = (
    3=>"PRECIPITACAO TOTAL, HORARIA (mm)",
    5=>"PRESSAO ATMOSFERICA MAX.NA HORA ANT.(AUT) (mB)",
    6=>"PRESSAO ATMOSFERICA MIN.NA HORA ANT.(AUT) (mB)",
    7=>"RADIACAO GLOBAL (KJ/m²)",
    8=>"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",
    9=>"TEMPERATURA DO PONTO DE ORVALHO (°C)",
    10=>"TEMPERATURA MAXIMA NA HORA ANT.(AUT) (°C)",
    11=>"TEMPERATURA MINIMA NA HORA ANT.(AUT) (°C)",
    12=>"TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)",
    13=>"TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)",
    17=>"VENTO, DIRECAO HORARIA (gr) (°(gr))"
)
for idx_ano in 1:length([2010, 2015, 2020])
    for col in columns_change
       rename!(dados.serie[idx_ano].dataset, col)
    end
end

In [17]:
println("Serie do ano $(dados.serie[1].ano) para a cidade de $(dados.serie[1].cidade)")
dados.serie[1].dataset

Serie do ano 2010 para a cidade de FORMIGA


,DATA (YYYY-MM-DD),HORA (UTC),"PRECIPITACAO TOTAL, HORARIA (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)"
,Date,Time,Float64,Float64
1,2010-01-01,00:00:00,0.0,912.6
2,2010-01-01,01:00:00,0.0,913.5
3,2010-01-01,02:00:00,0.0,913.8
4,2010-01-01,03:00:00,0.0,913.3
5,2010-01-01,04:00:00,0.0,912.8
6,2010-01-01,05:00:00,0.0,912.6
7,2010-01-01,06:00:00,0.0,912.0
8,2010-01-01,07:00:00,0.0,912.0
9,2010-01-01,08:00:00,0.0,912.2


In [18]:
println("Serie do ano $(dados.serie[2].ano) para a cidade de $(dados.serie[2].cidade)")
dados.serie[2].dataset

Serie do ano 2015 para a cidade de FORMIGA


,DATA (YYYY-MM-DD),HORA (UTC),"PRECIPITACAO TOTAL, HORARIA (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)"
,Date,Time,Float64,Float64
1,2015-01-01,00:00:00,0.0,916.2
2,2015-01-01,01:00:00,0.0,916.8
3,2015-01-01,02:00:00,0.0,916.5
4,2015-01-01,03:00:00,0.0,916.1
5,2015-01-01,04:00:00,0.0,915.7
6,2015-01-01,05:00:00,0.0,915.2
7,2015-01-01,06:00:00,0.0,915.1
8,2015-01-01,07:00:00,0.0,915.2
9,2015-01-01,08:00:00,0.0,915.6


In [19]:
println("Serie do ano $(dados.serie[3].ano) para a cidade de $(dados.serie[3].cidade)")
dados.serie[3].dataset

Serie do ano 2020 para a cidade de FORMIGA


,Data,Hora UTC,"PRECIPITACAO TOTAL, HORARIA (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)"
,String15,String15,Float64?,Float64?
1,2020/01/01,0000 UTC,0.0,912.4
2,2020/01/01,0100 UTC,0.0,912.7
3,2020/01/01,0200 UTC,0.0,913.2
4,2020/01/01,0300 UTC,0.0,913.0
5,2020/01/01,0400 UTC,0.0,913.0
6,2020/01/01,0500 UTC,0.0,912.8
7,2020/01/01,0600 UTC,0.0,912.8
8,2020/01/01,0700 UTC,0.0,913.2
9,2020/01/01,0800 UTC,0.0,913.2


### Aquisição de dados para múltiplos períodos

Como existe alta probabilidade, principalmente por tratar-se de séries temporais, que as aquisições se refiram a períodos sequenciais (adjacentes) a determinação do período total também pode ser realizada via `UnitRange` (vide: 
[https://docs.julialang.org/en/v1/base/collections/#Base.UnitRange]). Nestes casos o parâmetro `intervtemp` pode ser escrito como uma sequência de forma bastante reduzida. Veja o exemplo:

In [20]:
interv_temp = 2010:2014 #intervado de dados entre 2010 à 2014
dados = obter_dados(fonte_dados, ["FORMIGA"], interv_temp); 
columns_change = (
    3=>"PRECIPITACAO TOTAL, HORARIA (mm)",
    5=>"PRESSAO ATMOSFERICA MAX.NA HORA ANT.(AUT) (mB)",
    6=>"PRESSAO ATMOSFERICA MIN.NA HORA ANT.(AUT) (mB)",
    7=>"RADIACAO GLOBAL (KJ/m²)",
    8=>"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",
    9=>"TEMPERATURA DO PONTO DE ORVALHO (°C)",
    10=>"TEMPERATURA MAXIMA NA HORA ANT.(AUT) (°C)",
    11=>"TEMPERATURA MINIMA NA HORA ANT.(AUT) (°C)",
    12=>"TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)",
    13=>"TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)",
    17=>"VENTO, DIRECAO HORARIA (gr) (°(gr))"
)
for idx_ano in 1:length(interv_temp)
    for col in columns_change
       rename!(dados.serie[idx_ano].dataset, col)
    end
end

Excessive output truncated after 524334 bytes.

Os dados podem então ser acessados diretamente pelo campo `dataset`. Aqui usa-se a função `first` para exibir apenas as primeiras cinco linhas de cada série. 

In [21]:
println("Serie do ano $(dados.serie[1].ano) para a cidade de $(dados.serie[1].cidade)")
first(dados.serie[1].dataset, 5)

Serie do ano 2010 para a cidade de FORMIGA


,DATA (YYYY-MM-DD),HORA (UTC),"PRECIPITACAO TOTAL, HORARIA (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)"
,Date,Time,Float64,Float64
1,2010-01-01,00:00:00,0.0,912.6
2,2010-01-01,01:00:00,0.0,913.5
3,2010-01-01,02:00:00,0.0,913.8
4,2010-01-01,03:00:00,0.0,913.3
5,2010-01-01,04:00:00,0.0,912.8


In [22]:
println("Serie do ano $(dados.serie[2].ano) para a cidade de $(dados.serie[2].cidade)")
first(dados.serie[2].dataset, 5)

Serie do ano 2011 para a cidade de FORMIGA


,DATA (YYYY-MM-DD),HORA (UTC),"PRECIPITACAO TOTAL, HORARIA (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)"
,Date,Time,Float64,Float64
1,2011-01-01,00:00:00,0.2,910.6
2,2011-01-01,01:00:00,1.0,910.6
3,2011-01-01,02:00:00,0.0,911.1
4,2011-01-01,03:00:00,0.0,911.0
5,2011-01-01,04:00:00,0.2,910.4


In [23]:
println("Serie do ano $(dados.serie[3].ano) para a cidade de $(dados.serie[3].cidade)")
first(dados.serie[3].dataset, 5)

Serie do ano 2012 para a cidade de FORMIGA


,DATA (YYYY-MM-DD),HORA (UTC),"PRECIPITACAO TOTAL, HORARIA (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)"
,Date,Time,Float64,Float64
1,2012-01-01,00:00:00,1.8,911.9
2,2012-01-01,01:00:00,3.8,911.7
3,2012-01-01,02:00:00,1.2,911.5
4,2012-01-01,03:00:00,1.0,910.8
5,2012-01-01,04:00:00,0.6,910.1


In [24]:
println("Serie do ano $(dados.serie[4].ano) para a cidade de $(dados.serie[4].cidade)")
first(dados.serie[4].dataset, 4)

Serie do ano 2013 para a cidade de FORMIGA


,DATA (YYYY-MM-DD),HORA (UTC),"PRECIPITACAO TOTAL, HORARIA (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)"
,Date,Time,Float64,Float64
1,2013-01-01,00:00:00,0.0,915.3
2,2013-01-01,01:00:00,0.0,915.8
3,2013-01-01,02:00:00,0.0,916.0
4,2013-01-01,03:00:00,0.0,915.8


In [25]:
println("Serie do ano $(dados.serie[5].ano) para a cidade de $(dados.serie[5].cidade)")
first(dados.serie[5].dataset, 5)

Serie do ano 2014 para a cidade de FORMIGA


,DATA (YYYY-MM-DD),HORA (UTC),"PRECIPITACAO TOTAL, HORARIA (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)"
,Date,Time,Float64,Float64
1,2014-01-01,00:00:00,0.0,915.6
2,2014-01-01,01:00:00,0.0,916.2
3,2014-01-01,02:00:00,0.0,915.8
4,2014-01-01,03:00:00,0.0,915.6
5,2014-01-01,04:00:00,0.0,915.1


**Autor:** Reginaldo Gonçalves Leão Junior (prof.reginaldo.leao@gmail.com)

*Todos os direitos reservados*